In [6]:
import requests
import zipfile
import io
import os
import pandas as pd

# 1. 从网站下载数据
url = 'https://data.ptv.vic.gov.au/downloads/gtfs.zip'
response = requests.get(url)

# 2. 解压下载的zip文件
with zipfile.ZipFile(io.BytesIO(response.content)) as the_zip:
    the_zip.extractall('/Users/daixinxiu/Documents/GitHub/project-2-group-real-estate-industry-project-38/data/landing/stops_datavic')

# 3. 解压子文件夹中的所有zip文件
extracted_folder_path = '/Users/daixinxiu/Documents/GitHub/project-2-group-real-estate-industry-project-38/data/landing/stops_datavic'
for root, dirs, files in os.walk(extracted_folder_path):
    for file in files:
        if file.endswith('.zip'):
            file_path = os.path.join(root, file)
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(root)

# 4. 合并所有stops.txt文件为一个CSV
stops_files = []
for root, dirs, files in os.walk(extracted_folder_path):
    for file in files:
        if file == 'stops.txt':
            file_path = os.path.join(root, file)
            df = pd.read_csv(file_path)
            stops_files.append(df)

# 将所有DataFrame合并为一个
combined_stops_df = pd.concat(stops_files, ignore_index=True)

# 保存为一个新的CSV文件
output_csv_path = '/Users/daixinxiu/Documents/GitHub/project-2-group-real-estate-industry-project-38/data/raw/stops_datavic.csv'
combined_stops_df.to_csv(output_csv_path, index=False)


/var/folders/kt/yhd0mp_91gn1qrk_l__r080r0000gn/T/ipykernel_8704/149791997.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_stops_df = pd.concat(stops_files, ignore_index=True)


In [7]:
import pandas as pd
import re

def process_stops_csv(input_csv, output_csv):
    # 读取CSV文件
    df = pd.read_csv(input_csv)
    
    # 删除stop_id重复的行，并获取删除的数量
    before_count = len(df)
    df = df.drop_duplicates(subset=['stop_id'])
    after_count = len(df)
    deleted_count = before_count - after_count
    
    # 输出删除的数量
    print(f"Number of rows deleted due to duplicate stop_id: {deleted_count}")
    
    # 提取stop_name中的Region信息
    df['Region'] = df['stop_name'].apply(lambda x: re.search(r'\((.*?)\)', x).group(1) if re.search(r'\((.*?)\)', x) else 'Unknown')
    
    # 将处理后的DataFrame保存到新的CSV文件
    df.to_csv(output_csv, index=False)
    
    return deleted_count

# 示例使用
input_csv = '/Users/daixinxiu/Documents/GitHub/project-2-group-real-estate-industry-project-38/data/raw/stops_datavic.csv'

output_csv = '/Users/daixinxiu/Documents/GitHub/project-2-group-real-estate-industry-project-38/data/curated/stops_datavic.csv'


deleted_rows = process_stops_csv(input_csv, output_csv)


Number of rows deleted due to duplicate stop_id: 501
